# NOTEBOOK EDA

## Guión

### -Comprobar tipología de datos (intentar en el primer ntbook y si no en esté)

### -Listas de variables por tipo

### -Sacar gráficos

### -Estadísticos descriptivos de las variables

### -Correlaciones

### -Preguntas:

#### *Falsos Positivos y Falsos Negativos por variables demográficas
#### *Precisión del test (Porcentaje de aciertos)
#### *Tablas de contigencias de variables demográficas sobre is_recid y v_is_recid y score intervalos

In [ ]:
def eda(df):
    lista_variables_numericas = df.select_dtypes(include = "number").columns.tolist()
    lista_variables_categoricas = df.select_dtypes(include = ["object", "category"]).columns.tolist()
    lista_variables_temporales = df.select_dtypes(include = "datetime").columns.tolist()

    lista_a_eliminar = ["Nombre", "Apellido"]

    lista_variables_categoricas = [i for i in lista_variables_categoricas if i not in lista_a_eliminar]

    condicion = df["Género"] == "Femenino"
    print(f"Tamaño del dataframe con género femenino:\n{df[condicion].shape}")
    print("\n")
    print(f"Cantidad de categorías por género:\n{df.groupby("Género")["Categoría"].nunique()}")

    df["País"].value_counts().plot(kind = "bar", figsize = (15, 5))
    plt.title("Medallas por país")
    plt.show()
    print("\n")

    
    print(f"Media:\n{df[lista_variables_numericas].mean()}")
    print("\n")
    print(f"Mediana:\n{df[lista_variables_numericas].median()}")
    print("\n")
    print(f"Mínimo:\n{df[lista_variables_numericas].min()}")
    print("\n")
    print(f"Máximo:\n{df[lista_variables_numericas].max()}")
    print("\n")

    for col in lista_variables_numericas:
        df[[col]].boxplot(figsize=(8,5))
        plt.title(f"Distribución general de {col}")
        plt.show()
        print("\n")

    for col in lista_variables_numericas:

        sns.boxplot(
            data= df,
            x="Género",
            y= col,
            hue="Medalla"
        )
        plt.title(f"Distribución de {col} por género y medalla")
        plt.show()
        print("\n")

    for c in lista_variables_numericas:
        cat = sns.catplot(
        data=df,
        x="Categoría",
        y=c,
        col="Género",
        kind="box",
        sharey=False,
        height=4,
        aspect=1.
        )

        for ax in cat.axes.flat:
            ax.tick_params(axis="x", rotation = 45)
            ax.set_xticklabels(ax.get_xticklabels(), ha="right")

        cat.figure.suptitle(f"Distribución de {c} por categoría de peso", y=1.05, fontsize=14 )


    cat.figure.subplots_adjust(top=0.85)
    plt.show()
    print("\n")

    df["Año"] = df["Fecha"].dt.year.astype(int)

    medallas_pais = (
    df.groupby(["Año", "País"])
      .size()
      .reset_index(name="Medallas")
    )

    orden_paises = (
        medallas_pais.sort_values("Año")
                    .groupby("País")["Medallas"]
                    .last()
                    .sort_values(ascending=False)
                    .index
    )

    fig, ax = plt.subplots(figsize=(12, 10))

    sns.lineplot(
        data=medallas_pais,
        x="Año",
        y="Medallas",
        hue="País",
        hue_order=orden_paises,
        marker="o",
        palette="tab20",
        ax=ax
    )

    ax.legend(
        title="País",
        bbox_to_anchor=(1.02, 1),
        loc="upper left",
        borderaxespad=0
    )

    ax.set_title("Evolución de medallas por país")
    ax.set_xticks(sorted(df["Año"].unique()))
    ax.set_xticklabels(sorted(df["Año"].unique()))

    fig.subplots_adjust(right=0.8)

    plt.show()


    print("\n")

    for var in lista_variables_numericas:
        sns.lineplot(
            data=df,
            x="Año",
            y=var,
            hue="Género",
            marker="o"
        )
        plt.title(f"Evolución temporal de {var}")
        plt.xticks(df["Año"].unique())
        plt.show()
        print("\n")

    sns.scatterplot(
    data= df,
    x="Arrancada",
    y="Dos tiempos",
    hue="Género"
    )

    min_val = min(df["Arrancada"].min(), df["Dos tiempos"].min())
    max_val = max(df["Arrancada"].max(), df["Dos tiempos"].max())


    plt.xlabel("Arrancada")
    plt.ylabel("Dos tiempos")
    plt.title(f"Correlacción entre arrancada y dos tiempos")
    plt.show()
    print("\n")

    sns.scatterplot(
    data= df,
    x="Total",
    y="Arrancada",
    hue="Género"
    )

    min_val = min(df["Total"].min(), df["Arrancada"].min())
    max_val = max(df["Total"].max(), df["Arrancada"].max())

    plt.xlabel("Total")
    plt.ylabel("Arrancada")
    plt.title(f"Correlacción entre levantamiento en arrancada y el total de peso levantado")
    plt.show()
    print("\n")

    sns.scatterplot(
    data= df,
    x="Total",
    y="Dos tiempos",
    hue="Género"
    )

    min_val = min(df["Total"].min(), df["Dos tiempos"].min())
    max_val = max(df["Total"].max(), df["Dos tiempos"].max())

    plt.xlabel("Total")
    plt.ylabel("Dos tiempos")
    plt.title(f"Correlacción entre levantamiento en dos tiempos y el total de peso levantado")
    plt.show()
    print("\n")

    print("Correlación entre levantamientos")
    display(df[lista_variables_numericas].corr())